In [1]:
from __future__ import print_function

import cobra
import cobra.test
import mackinac
import numpy as np
import csv
import glob
import pickle

In [3]:
mackinac.get_token('tjmoutinho')

patric password: ········


u'tjmoutinho@patricbrc.org'

In [2]:
# Read in list of all genomes on PATRIC and list of existing models in folder
with open('../Data/genome_ids_all.csv') as csvfile:
    genome_ids_list = []
    for line in csvfile:
        genome_ids_list.append(line.strip())
# genome_ids_list
models = glob.glob('../models/*.xml')
models = [x.replace("../models/","").replace(".xml","") for x in models]
# len(genome_ids_list)

In [4]:
# Check for duplicates
len(models) == len(set(models))

True

In [ ]:
# Download additional models and convert to cobra model, then save to sbml
failed_genome_index = []
for j in range(10):
    for genome_id in genome_ids_list:
        if not genome_id in models:
            try:
                model_file_name = "models/%s.xml" % (genome_id)
                mackinac.create_patric_model(genome_id,genome_id)
                model = mackinac.create_cobra_model_from_patric_model(genome_id)
                cobra.io.write_sbml_model(model, model_file_name)
                genome_ids_list.remove(genome_id)
            except:
                failed_genome_index.append(genome_id)
                pass

In [ ]:
# Download fasta files

# Check for existing fasta files by name
fasta_file_paths = glob.glob('fastas/*.faa')
fasta_genome_ids = [x.replace("fastas/","").replace(".faa","") for x in fasta_file_paths]
# Loop to make all missing Fasta Files
failed_genome_fasta_index = []
for genome_id in genome_ids_list:
    if not genome_id in fasta_genome_ids:
        try:
            feature_list = mackinac.get_genome_features(genome_id, annotation='PATRIC')
            file_name = "fastas/%s.faa" % (genome_id)
            feats = mackinac.features_to_protein_fasta_file(feature_list, file_name)
            #print("Number of Features in genome " + str(x) + ": " + str(feats))
        except:
            failed_genome_fasta_index.append(genome_id)
            pass

In [ ]:
# Download features files

# Check for existing fasta files by name
feature_file_paths = glob.glob('features/*.feats')
feature_genome_ids = [x.replace("features/","").replace(".feats","") for x in feature_file_paths]
# Loop to make all missing Fasta Files
missing_feat_files = []
for genome_id in genome_ids_list:
    if not genome_id in feature_genome_ids:
        try:
            feature_list = mackinac.get_genome_features(genome_id, annotation='PATRIC')
            file_name = "features/%s.feats" % (genome_id)
            pickle.dump(feature_list, open(file_name, "wb"))
        except:
            missing_feat_files.append(genome_id)
            pass